# COMPUTATIONAL THEORY PROBLEMS

In [118]:
# IMPORTS
import numpy as np

## Problem 1: Binary Words and Operations
SHA-256 uses seven core bitwise functions that operate on 32-bit words. These functions manipulate individual bits using logical operations (```AND, OR, XOR, NOT```) and bit shifts/rotations. All operations must be performed with 32-bit arithmetic to match the SHA-256 specification.

### 1.1 Parity Function - computes the bitwise XOR of three 32-bit words

**How this function works:** It takes three integers and converts them to 32-bit integers (``` np.uint32 ```). It performs ```XOR``` operations which compare bits pairwise, returning 1 when bits are different and 0 when they're the same.

In [119]:
def Parity(x, y, z):
    """
    Parity function [see 1, p.10, eq. 4.1].
    
    Returns the XOR of three 32-bit words: x ⊕ y ⊕ z
    
    For each bit position, returns 1 if an odd number of inputs 
    have a 1 bit at that position.
    
    Args:
        x, y, z: 32-bit unsigned integers
        
    Returns:
        32-bit unsigned integer
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    return x ^ y ^ z


In [120]:
# Test Parity function
print("Testing Parity:")

# Test 1: Basic XOR property
print(f"Parity(0xF, 0xF, 0xF) = {Parity(0xF, 0xF, 0xF):#x}")  # Should be 0xF

# Test 2: Cancellation (x ⊕ y ⊕ y = x)
print(f"Parity(0xABCD, 0x1234, 0x1234) = {Parity(0xABCD, 0x1234, 0x1234):#x}")  # Should be 0xABCD

# Test 3: All zeros
print(f"Parity(0x0, 0x0, 0x0) = {Parity(0x0, 0x0, 0x0):#x}")  # Should be 0x0

# Test 4: All ones
print(f"Parity(0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF) = {Parity(0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF):#010x}")  # Should be 0xFFFFFFFF

Testing Parity:
Parity(0xF, 0xF, 0xF) = 0xf
Parity(0xABCD, 0x1234, 0x1234) = 0xabcd
Parity(0x0, 0x0, 0x0) = 0x0
Parity(0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF) = 0xffffffff


### 1.2 Ch (Choose) Function - Select bits from y or z based on x

**How this function works:** The idea here is the same as in the Parity function (see 1.1) but instead of only doing ```XOR``` operations we first do an ```AND``` operation between x and y, then we do an ```AND``` operation between the ```NOT``` of x and z. Finally, we ```XOR``` these two results together. The ```AND``` operations act as filters that select which bits to keep from y and z, while the ```XOR``` combines them into the final result. 

```(x & y)``` is just a basic ```AND``` operation (if both bits are 1 it returns 1). 

```(~x & z)``` flips all bits in x before carrying out the ```AND``` operation with z.

We then XOR the result of these two operations.

In [121]:
def Ch(x, y, z):
    """
    Choose function [see 1, p.10, eq. 4.2].

    Bitwise choice function for SHA-256.
    
    For each bit position, returns the bit from y if the corresponding
    bit in x is 1, otherwise returns the bit from z

    Args:
        x, y, z: 32-bit unsigned integers

    Returns:
        32-bit unsigned integer result of the bitwise choice operation
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    return (x & y) ^ (~x & z)

In [122]:
# Test Ch function
print("Testing Ch:")

# Test 1: All 1s in x selects -> y entirely
print(f"Ch(0xFFFFFFFF, 0xAAAAAAAA, 0x55555555) = {Ch(0xFFFFFFFF, 0xAAAAAAAA, 0x55555555):#010x}")  # Should be 0xAAAAAAAA

# Test 2: All 0s in x selects -> z entirely
print(f"Ch(0x00000000, 0xAAAAAAAA, 0x55555555) = {Ch(0x00000000, 0xAAAAAAAA, 0x55555555):#010x}")  # Should be 0x55555555

# Test 3: Mixed selector - alternates between y and z
print(f"Ch(0xF0F0F0F0, 0xFFFFFFFF, 0x00000000) = {Ch(0xF0F0F0F0, 0xFFFFFFFF, 0x00000000):#010x}")  # Should be 0xF0F0F0F0

Testing Ch:
Ch(0xFFFFFFFF, 0xAAAAAAAA, 0x55555555) = 0xaaaaaaaa
Ch(0x00000000, 0xAAAAAAAA, 0x55555555) = 0x55555555
Ch(0xF0F0F0F0, 0xFFFFFFFF, 0x00000000) = 0xf0f0f0f0


### 1.3 Maj (Majority) Function - Returns the value that appears in at least 2 out of 3 inputs

**How this function works:** We perform three ```AND``` operations to find where pairs of inputs are the same (`x & y`, `x & z`, `y & z`), then ```XOR``` these results together. This produces 1 at positions where at least two inputs have 1.

In [123]:
def Maj(x, y, z):
    """
    Majority function [see 1, p.10, eq. 4.3].
    Used in SHA-256 compression function [see 1, p.22-23].
    
    For each bit position, returns the bit value that appears 
    in at least 2 of the 3 inputs (the majority).
    
    Args:
        x, y, z: 32-bit unsigned integers
        
    Returns:
        32-bit unsigned integer
    """
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    return (x & y) ^ (x & z) ^ (y & z)

In [124]:
# Test Maj function
print("Testing Maj:")

# Test 1: Two inputs the same
print(f"Maj(0xFFFFFFFF, 0xFFFFFFFF, 0x00000000) = {Maj(0xFFFFFFFF, 0xFFFFFFFF, 0x00000000):#010x}")  # Should be 0xFFFFFFFF (majority is 1)

# Test 2: Two inputs the same (other pattern)
print(f"Maj(0x00000000, 0xFFFFFFFF, 0xFFFFFFFF) = {Maj(0x00000000, 0xFFFFFFFF, 0xFFFFFFFF):#010x}")  # Should be 0xFFFFFFFF

# Test 3: All same
print(f"Maj(0xAAAAAAAA, 0xAAAAAAAA, 0xAAAAAAAA) = {Maj(0xAAAAAAAA, 0xAAAAAAAA, 0xAAAAAAAA):#010x}")  # Should be 0xAAAAAAAA

Testing Maj:
Maj(0xFFFFFFFF, 0xFFFFFFFF, 0x00000000) = 0xffffffff
Maj(0x00000000, 0xFFFFFFFF, 0xFFFFFFFF) = 0xffffffff
Maj(0xAAAAAAAA, 0xAAAAAAAA, 0xAAAAAAAA) = 0xaaaaaaaa


### 1.4 - 1.7 Rotation and Shift Operations

**How these functions work:** These four functions use rotation (ROTR) and shift (SHR) operations instead of logical operations. 

**ROTR^n(x)** (Rotate Right): Moves all bits n positions to the right. Bits that fall off the right side are placed back in the left side. No bits are lost.

**SHR^n(x)** (Shift Right): Moves all bits n positions to the right. Bits that fall off the right are lost, and we fill the empty slots with zeros.

**Functions 1.4-1.5 (Σ₀ and Σ₁):** Use only ROTR operations.

**Functions 1.6-1.7 (σ₀ and σ₁):** Use both ROTR and SHR operations.

In [125]:
# Rotate Right
def ROTR(x, n):
    """
    Rotate right (circular right shift).
    
    ROTR^n(x) = (x >> n) ∨ (x << (32 - n))
    
    Moves bits n positions to the right, placing bits that 
    fall off back to the left side.
    
    Args:
        x: 32-bit unsigned integer
        n: number of positions to rotate (needs to be in range 0-31 and >= 0)
        
    Returns:
        32-bit unsigned integer
    """
    x = np.uint32(x)
    
    return (x >> n) | (x << (32 - n))

In [126]:
# Shift Right
def SHR(x, n):
    """
    Shift right.
    
    SHR^n(x) = x >> n (shift by n times)
    
    Moves bits n positions to the right, filling left side with zeros.
    Bits that fall off the right get lost.
    
    Args:
        x: 32-bit unsigned integer
        n: number of positions to shift (0 ≤ n < 32)
        
    Returns:
        32-bit unsigned integer
    """
    x = np.uint32(x)
    
    return x >> n

### 1.4 Sigma0 (Σ₀)

Combines three rotations of x (ROTR by 2, 13, and 22 bits in this case) using ```XOR```. Creates diffusion by mixing bits from different positions.

In [127]:
def Sigma0(x):
    """
    Sigma0 function [see 1, p.10, eq. 4.4].
    Used in SHA-256 compression function to calculate T₂ [see 1, p.22-23].
    
    Σ₀(x) = ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x)
    
    Args:
        x: 32-bit unsigned integer
        
    Returns:
        32-bit unsigned integer
    """
    x = np.uint32(x)
    
    return ROTR(x, 2) ^ ROTR(x, 13) ^ ROTR(x, 22)

In [128]:
# Test Sigma0
print("Testing Sigma0:")
print(f"Sigma0(0x00000000) = {Sigma0(0x00000000):#010x}")  # Should be 0x00000000
print(f"Sigma0(0xFFFFFFFF) = {Sigma0(0xFFFFFFFF):#010X}")  # Should be 0xFFFFFFFF
print(f"Sigma0(0x12345678) = {Sigma0(0x12345678):#010x}")  # Real scrambling testing

Testing Sigma0:
Sigma0(0x00000000) = 0x00000000
Sigma0(0xFFFFFFFF) = 0XFFFFFFFF
Sigma0(0x12345678) = 0x66146474


### 1.5 Sigma1 (Σ₁)

Combines three rotations of x (ROTR by 6, 11, and 25 bits) using ```XOR```. Same structure as Sigma0 but with different rotation amounts.

In [129]:
def Sigma1(x):
    """
    Sigma1 function [see 1, p.10, eq. 4.5].
    Used in SHA-256 compression function to calculate T₁ [see 1, p.22-23].

    Args:
        x: 32-bit unsigned integer
        
    Returns:
        32-bit unsigned integer
    """
    x = np.uint32(x)
    
    return ROTR(x, 6) ^ ROTR(x, 11) ^ ROTR(x, 25)

In [130]:
# Test Sigma1
print("Testing Sigma1:")
print(f"Sigma1(0x00000000) = {Sigma1(0x00000000):#010x}")  # Should be 0x00000000
print(f"Sigma1(0xFFFFFFFF) = {Sigma1(0xFFFFFFFF):#010x}")  # Should be 0xFFFFFFFF
print(f"Sigma1(0x12345678) = {Sigma1(0x12345678):#010x}")  # Real scrambling testing

Testing Sigma1:
Sigma1(0x00000000) = 0x00000000
Sigma1(0xFFFFFFFF) = 0xffffffff
Sigma1(0x12345678) = 0x3561abda


### 1.6 sigma0 (σ₀)

Combines two rotations (by 7 and 18 bits) and one shift (by 3 bits) using ```XOR```. The shift operation means some bits are lost, unlike the uppercase Sigma functions.

In [131]:
def sigma0(x):
    """
    sigma0 function [see 1, p.10, eq. 4.6].
    Used in message schedule expansion [see 1, p.22].
    
    Args:
        x: 32-bit unsigned integer
        
    Returns:
        32-bit unsigned integer
    """
    x = np.uint32(x)
    
    return ROTR(x, 7) ^ ROTR(x, 18) ^ SHR(x, 3)

### 1.7 sigma1 (σ₁)

Combines two rotations (by 17 and 19 bits) and one shift (by 10 bits) using ```XOR```. Similar to sigma0, uses shift which loses bits.

In [132]:
def sigma1(x):
    """
    sigma1 function [see 1, p.10, eq. 4.7].
    Used in message schedule expansion [see 1, p.22, step 1].
    
    Args:
        x: 32-bit unsigned integer
        
    Returns:
        32-bit unsigned integer
    """
    x = np.uint32(x)
    
    return ROTR(x, 17) ^ ROTR(x, 19) ^ SHR(x, 10)

In [133]:
# Test sigma0 and sigma1
print("Testing sigma0:")
print(f"sigma0(0x00000000) = {sigma0(0x00000000):#010x}")  # Should be 0x00000000
print(f"sigma0(0xFFFFFFFF) = {sigma0(0xFFFFFFFF):#010x}")  # Should be 0x1FFFFFFF
print(f"sigma0(0x12345678) = {sigma0(0x12345678):#010x}")  # Shows scrambling

print("\nTesting sigma1:")
print(f"sigma1(0x00000000) = {sigma1(0x00000000):#010x}")  # Should be 0x00000000
print(f"sigma1(0xFFFFFFFF) = {sigma1(0xFFFFFFFF):#010x}")  # Should be 0x003FFFFF
print(f"sigma1(0x12345678) = {sigma1(0x12345678):#010x}")  # Shows scrambling

Testing sigma0:
sigma0(0x00000000) = 0x00000000
sigma0(0xFFFFFFFF) = 0x1fffffff
sigma0(0x12345678) = 0xe7fce6ee

Testing sigma1:
sigma1(0x00000000) = 0x00000000
sigma1(0xFFFFFFFF) = 0x003fffff
sigma1(0x12345678) = 0xa1f78649


### Summary of Problem 1 Functions

| Function | Type | Inputs | Operations | Used In |
|----------|------|--------|------------|---------|
| Parity | Logical | 3 | XOR | SHA-1 |
| Ch | Logical | 3 | AND, NOT, XOR | Compression |
| Maj | Logical | 3 | AND, XOR | Compression |
| Σ₀ | Rotation | 1 | ROTR×3, XOR | Compression (T₂) |
| Σ₁ | Rotation | 1 | ROTR×3, XOR | Compression (T₁) |
| σ₀ | Mixed | 1 | ROTR×2, SHR×1, XOR | Message schedule |
| σ₁ | Mixed | 1 | ROTR×2, SHR×1, XOR | Message schedule |

## Problem 2: Fractional Parts of Cube Roots

This problem aims to create the 64 "magic numbers" that SHA-512 uses internally. These come from cube roots of the first 64 prime numbers [see 1, p.11], which ensures they're mathematically verifiable and not just random values. We'll showcase how to get to these values by following the steps below.

**Steps:**
1. Write a function `primes(n)` that generates the first n prime numbers
2. Use the function to calculate the cube root of the first 64 primes
3. For each cube root, extract the first 32 bits of the fractional part
4. Display the result in hexadecimal
5. Test the results against what is in the Secure Hash Standard

In [134]:
# 1. Function to generate first n prime numbers
def primes(n):
    """Generate the first n prime numbers."""
    # If n is less than 1, return empty list
    if n < 1:
        return []
    
    prime_list = []
    num = 2  # 2 is the first prime number
    
    # Keep finding primes until we have n of them
    while len(prime_list) < n:
        is_prime = True  # Assume the number is prime
        
        # Check if num is divisible by any number from 2 to num-1
        for i in range(2, num):
            if num % i == 0:  # If true, it's not prime
                is_prime = False
                break  # Stop here
        
        # If num passed all checks, it's prime, add it to our list
        if is_prime:
            prime_list.append(num)
        
        num += 1  # Move to the next number
    
    return prime_list


In [135]:
# 2. Calculate cube roots of first 64 primes
first_64_primes = primes(64)

# Calculate cube roots using numpy
cube_roots = np.cbrt(first_64_primes)

In [136]:
# Test prime generation and cube roots
print("First 64 primes:", first_64_primes)
print("Cube roots of first 64 primes:", cube_roots)

First 64 primes: [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311]
Cube roots of first 64 primes: [1.25992105 1.44224957 1.70997595 1.91293118 2.22398009 2.35133469
 2.57128159 2.66840165 2.84386698 3.07231683 3.14138065 3.33222185
 3.44821724 3.50339806 3.60882608 3.75628575 3.89299642 3.93649718
 4.0615481  4.14081775 4.1793392  4.29084043 4.36207067 4.4647451
 4.59470089 4.65700951 4.68754815 4.7474594  4.77685618 4.83458813
 5.0265257  5.07875308 5.15513674 5.18010147 5.30145919 5.32507402
 5.39469071 5.46255557 5.50687845 5.57205466 5.63574079 5.65665283
 5.75896522 5.77899657 5.81864787 5.83827246 5.95334181 6.06412699
 6.1001702  6.11803317 6.15344949 6.20582179 6.22308425 6.30799355
 6.35786118 6.40695858 6.45531481 6.47127363 6.51868392 6.

In [137]:
# 3. Extract first 32 bits of fractional part of each cube root
constants = []  # Store the 32-bit constants

for root in cube_roots:
    # Get everything after the decimal point
    # If root = 1.2599, fractional_part = 0.2599
    fractional_part = root - int(root)
    
    # Shift left by 32 bits (multiply by 2^32)
    # This moves the first 32 bits of the fraction into the integer part
    shifted = fractional_part * (2 ** 32)
    
    # Convert to integer to get just those 32 bits
    int_value = int(shifted)
    
    # Add to our list of constants
    constants.append(int_value)

print("First 5 constants (decimal):", constants[:5])


First 5 constants (decimal): [1116352408, 1899447441, 3049323471, 3921009573, 961987163]


## Problem 3: Padding

In [138]:
# Problem 3

## Problem 4: Hashes

In [139]:
# Problem 4

## Problem 5: Passwords

In [140]:
# Problem 5

## References

[1] National Institute of Standards and Technology. (2015). *Secure Hash Standard (SHS)*. FIPS PUB 180-4. Available at: https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf

[2] Wikipedia contributors. (2025). *SHA-2*. In Wikipedia, The Free Encyclopedia. Available at: https://en.wikipedia.org/wiki/SHA-2


# END